# Example: Tool Usage of Responses API

This example demonstrates how to use the Responses API to interact with a custom tool. The workflow includes defining a tool schema, handling function calls, and providing outputs back to the model. The example showcases a weather-fetching tool that responds to user queries about the weather in a specific location.

In [30]:
from dotenv import load_dotenv
from openai import OpenAI
import requests
import json

# Get OPENAI_API_KEY from .env file
load_dotenv()
client = OpenAI()

In [31]:
def get_weather(latitude, longitude):
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]["temperature_2m"]

In [32]:
# Define the tool's schema
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"},
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False,
        },
        "strict": True,
    }
]

In [33]:
input_messages = [
    {"role": "user", "content": "What's the weather like in Paris today?"}
]

In [34]:
response = client.responses.create(
    model="gpt-4o",
    input=input_messages,
    tools=tools,
)

In [35]:
response.output

[ResponseFunctionToolCall(arguments='{"latitude":48.8566,"longitude":2.3522}', call_id='call_sX7Jk3LcQCmCdCKgEY4id8Wr', name='get_weather', type='function_call', id='fc_67fcacc462ac8191b2d7fefa922b69550386f4b1b4b65eb8', status='completed')]

In [36]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = get_weather(args["latitude"], args["longitude"])

In [37]:
result

9.2

In [38]:
input_messages.append(tool_call)  # append model's function call message
input_messages.append(
    {  # append result message
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result),
    }
)

response_2 = client.responses.create(
    model="gpt-4o",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

The current temperature in Paris is 9.2°C.
